# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [37]:
# Your solution goes here

import requests
from bs4 import BeautifulSoup
import pandas as pd


# Función para convertir la clase de estrellas en un número entero
def get_rating(stars):
    rating_dict = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }
    return rating_dict.get(stars, 0)  # Devuelve el valor numérico correspondiente a la clase de estrellas

# Función para extraer los libros que cumplen con los criterios de búsqueda
def scrape_books(min_rating, max_price):
    base_url = "https://books.toscrape.com/catalogue/page-{}.html"  # URL base del sitio web con un marcador de página
    books = []  # Lista para almacenar los libros que cumplen con los criterios de búsqueda
    page = 1  # Iniciar en la primera página

    while True:
        try:
            print(f"Procesando página {page}...")  # Imprime el número de página que se está procesando
            url = base_url.format(page)  # Crea la URL de la página actual
            response = requests.get(url)  # Realiza una solicitud GET a la URL de la página actual
            response.raise_for_status()  # Lanza una excepción si la solicitud no fue exitosa
            
            soup = BeautifulSoup(response.content, 'html.parser')  # Analiza el contenido HTML de la página
            
            # Verificar si hay libros en la página actual
            books_on_page = soup.select('.product_pod')
            if not books_on_page:
                print("No hay más libros disponibles.")
                break
            
            # Itera sobre cada libro en la página actual
            for book in books_on_page:
                title = book.h3.a['title']  # Título del libro
                price = float(book.select_one('.price_color').text[1:])  # Precio del libro
                rating = get_rating(book.p['class'][1])  # Clase de estrellas convertida a un número entero
                availability = book.select_one('.availability').text.strip()  # Disponibilidad del libro
                book_url = "https://books.toscrape.com/catalogue/" + book.h3.a['href']  # URL del libro
                
                # Si el libro cumple con los criterios de búsqueda, se extraen más detalles
                if rating >= min_rating and price <= max_price:
                    try:
                        book_response = requests.get(book_url)  # Realiza una solicitud GET a la URL del libro
                        book_response.raise_for_status()  # Lanza una excepción si la solicitud no fue exitosa
                        
                        book_soup = BeautifulSoup(book_response.content, 'html.parser')  # Crea un objeto BeautifulSoup para el libro
                        
                        upc = book_soup.select_one('th:contains("UPC") + td').text  # Código UPC del libro
                        genre = book_soup.select_one('ul.breadcrumb li:nth-child(3) a').text  # Género del libro
                        description = book_soup.select_one('#product_description ~ p')  # Descripción del libro
                        description = description.text if description else "No description available"  # Si no hay descripción, se asigna un mensaje predeterminado
                        
                        # Añade los detalles del libro a la lista de libros
                        books.append({
                            "UPC": upc,
                            "Title": title,
                            "Price (£)": price,
                            "Rating": rating,
                            "Genre": genre,
                            "Availability": availability,
                            "Description": description
                        })
                    except requests.RequestException as e:
                        print(f"Error al acceder al libro {title}: {e}")
                        continue

            page += 1  # Incrementar el número de página para la siguiente iteración

        except requests.RequestException as e:
            print(f"Error al acceder a la página {page}: {e}")
            break

    # Crear un DataFrame de pandas con los libros que cumplen con los criterios de búsqueda
    df = pd.DataFrame(books)
    
    # Asegurar la consistencia de los tipos de datos
    df['Price (£)'] = df['Price (£)'].astype(float)
    df['Rating'] = df['Rating'].astype(int)
    
    # Manejar valores nulos o faltantes
    df.fillna("No data available", inplace=True)
    
    return df


In [38]:
# Ejemplo de uso
df = scrape_books(min_rating=4, max_price=20)

# Display the DataFrame
df

Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Procesando página 6...
Procesando página 7...
Procesando página 8...
Procesando página 9...
Procesando página 10...
Procesando página 11...
Procesando página 12...
Procesando página 13...
Procesando página 14...
Procesando página 15...
Procesando página 16...
Procesando página 17...
Procesando página 18...
Procesando página 19...
Procesando página 20...
Procesando página 21...
Procesando página 22...
Procesando página 23...
Procesando página 24...
Procesando página 25...
Procesando página 26...
Procesando página 27...
Procesando página 28...
Procesando página 29...
Procesando página 30...
Procesando página 31...
Procesando página 32...
Procesando página 33...
Procesando página 34...
Procesando página 35...
Procesando página 36...
Procesando página 37...
Procesando página 38...
Procesando página 39...
Procesando página 40...
Procesando página 41...
Procesando página 42...
P

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock,Aaron Ledbetter’s future had been planned out ...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock,A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,In stock,"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,In stock,"Every summer, Rose goes with her mom and dad t..."
...,...,...,...,...,...,...,...
70,9c96cd1329fbd82d,The Zombie Room,19.69,5,Default,In stock,An unlikely bond is forged between three men f...
71,b78deb463531d078,The Silent Wife,12.34,5,Fiction,In stock,A chilling psychological thriller about a marr...
72,4280ac3eab57aa5d,The Girl You Lost,12.29,5,Mystery,In stock,Eighteen years ago your baby daughter was snat...
73,29fc016c459aeb14,The Edge of Reason (Bridget Jones #2),19.18,4,Womens Fiction,In stock,Monday 27 January“7:15 a.m. Hurrah! The wilder...


In [40]:
# Filtrar libros con calificación mayor a 4
high_rated_books = df[df['Rating'] > 4]
print("Libros con calificación mayor a 4:")
high_rated_books

Libros con calificación mayor a 4:


,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock,Aaron Ledbetter’s future had been planned out ...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock,A page-turning novel that is also an explorati...
5,709822d0b5bcb7f4,Thirst,17.27,5,Fiction,In stock,"On a searing summer Friday, Eddie Chapman has ..."
6,0fa6dceead7ce47a,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Sequential Art,In stock,THE LONG-AWAITED STORY OF FANGIRLS TAKING ON T...
7,0e691eda369f4e09,Princess Between Worlds (Wide-Awake Princess #5),13.34,5,Fantasy,In stock,Just as Annie and Liam are busy making plans t...
12,3bebf34ee9330cbd,The Third Wave: An Entrepreneur’s Vision of th...,12.61,5,Business,In stock,NEW YORK TIMES BESTSELLERWALL STREET JOURNAL B...
16,88c21fcd38e2486e,Dark Notes,19.19,5,Erotica,In stock,They call me a slut. Maybe I am.Sometimes I do...
17,47271d8e08b5d31c,Batman: The Dark Knight Returns (Batman),15.38,5,Sequential Art,In stock,This masterpiece of modern comics storytelling...
18,d195e7f4ed697350,Agnostic: A Spirited Manifesto,12.51,5,Nonfiction,In stock,A widely admired writer on religion celebrates...


In [41]:
# Mostrar estadísticas de las columnas de precio y calificación
print("Estadísticas de las columnas de precio y calificación:")
print(df[['Price (£)', 'Rating']].describe())

Estadísticas de las columnas de precio y calificación:
       Price (£)    Rating
count  75.000000  75.00000
mean   14.570933   4.56000
std     2.728553   0.49973
min    10.000000   4.00000
25%    12.340000   4.00000
50%    14.440000   5.00000
75%    16.850000   5.00000
max    19.690000   5.00000
